In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2000&cat=FieldGoalsAttempted3&agg=PerGame"

In [3]:
for year in years:
    with open("attempted_threepointers_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [4]:
dfs = []
for year in years:
    with open("attempted_threepointers_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    attempted_threepointers   = pd.read_html(str(player_table))[0]
    attempted_threepointers["Year"] = year
    dfs.append(attempted_threepointers)

In [5]:
attempted_threepointers = pd.concat(dfs)
attempted_threepointers

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"SESAR, JOSIPSESAR, J.",CIBCibona VIPCibona VIP,12,8.17,2.00,24.49%,2001
1,2,"KUTLUAY, IBRAHIMKUTLUAY, I.",AEKAEK AthensAEK Athens,18,6.22,2.17,34.82%,2001
2,3,"BROWN, MARCUSBROWN, M.",TREBenetton BasketBenetton Basket,10,5.90,2.50,42.37%,2001
3,4,"NAUMOSKI, PETARNAUMOSKI, P.",TREBenetton BasketBenetton Basket,15,5.67,2.33,41.18%,2001
4,4,"WILLIAMS, HENRYWILLIAMS, H.",VERMuller VeronaMuller Verona,12,5.67,2.17,38.24%,2001
...,...,...,...,...,...,...,...,...
45,46,"FOIREST, LAURENTFOIREST, L.",BKNTau Ceramica Vitoria-GasteizTau Ceramica Vi...,21,3.33,1.29,38.57%,2021
46,47,"KENNEDY, TIMKENNEDY, TIM",OVAOvarense AerosolesOvarense Aerosoles,10,3.30,1.10,33.33%,2021
47,47,"KARNISOVAS, ARTURASKARNISOVAS, A.",FCBFC BarcelonaFC Barcelona,10,3.30,1.20,36.36%,2021
48,47,"BOWIE, ANTHONYBOWIE, A.",FORPAF BolognaPAF Bologna,10,3.30,1.50,45.45%,2021


In [6]:
engine = create_engine('sqlite:///attempted_threepointers_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
attempted_threepointers.to_sql(
    name = "attempted_threepointers_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [8]:
attempted_threepointers_leaders_select = session.execute('select * from "attempted_threepointers_leaders"')
attempted_threepointers_leaders_select.all()

[(0, 1, 'SESAR, JOSIPSESAR, J.', 'CIBCibona VIPCibona VIP', 12, 8.17, 2.0, '24.49%', 2001),
 (1, 2, 'KUTLUAY, IBRAHIMKUTLUAY, I.', 'AEKAEK AthensAEK Athens', 18, 6.22, 2.17, '34.82%', 2001),
 (2, 3, 'BROWN, MARCUSBROWN, M.', 'TREBenetton BasketBenetton Basket', 10, 5.9, 2.5, '42.37%', 2001),
 (3, 4, 'NAUMOSKI, PETARNAUMOSKI, P.', 'TREBenetton BasketBenetton Basket', 15, 5.67, 2.33, '41.18%', 2001),
 (4, 4, 'WILLIAMS, HENRYWILLIAMS, H.', 'VERMuller VeronaMuller Verona', 12, 5.67, 2.17, '38.24%', 2001),
 (5, 6, 'MYERS, CARLTONMYERS, C.', 'FORPAF BolognaPAF Bologna', 10, 5.6, 2.0, '35.71%', 2001),
 (6, 7, 'BULLOCK, LOUISBULLOCK, L.', 'VERMuller VeronaMuller Verona', 11, 5.45, 2.27, '41.67%', 2001),
 (7, 8, 'RUZIC, JURICARUZIC, J.', 'ZADZadarZadar', 10, 5.4, 1.9, '35.19%', 2001),
 (8, 9, 'MARIC, IVICAMARIC, I.', 'ZADZadarZadar', 10, 5.2, 2.1, '40.38%', 2001),
 (9, 10, 'BASILE, GIANLUCABASILE, G.', 'FORPAF BolognaPAF Bologna', 18, 5.0, 1.56, '31.11%', 2001),
 (10, 10, 'MRAZEK, HAROLDMRAZEK,